In [1]:
from llms.google_imagen_image_gen import google_imagen_image_generator

ai_response, filepath = google_imagen_image_generator("Bugs Bunny and Donald Duck shaking hands. Photorealistic, highly detailed")

print(f"\nFilepath: {filepath}")

ℹ️ Starting Google Imagen image generation

🐛 Input prompt: Bugs Bunny and Donald Duck shaking hands. Photorealistic, highly detailed

🐛 Initialized GoogleImagenImageGenerator with base URL: https://api.a4f.co/v1

🐛 Initialized GoogleImagenImageGenerator

ℹ️ Starting image generation and save process

...

ℹ️ Image saved to: images/generated_images/google_imagen_image_20250711_155314.png

✅ Image generation completed successfully

ℹ️ Image saved to: images/generated_images/google_imagen_image_20250711_155314.png

🐛 AI response: Generated image successfully for prompt: Bugs Bunny and Donald Duck shaking hands. Photorealistic, 
highly detailed


Filepath: images/generated_images/google_imagen_image_20250711_155314.png
